# GUI design tests

After adding a few buttons, it became clear that it's easy to write messy code for a GUI. A major difficulty are the numerous links between the graphical elements that contrasts to the desire to write reusable code. The default GUI elemenents in IPyWidgets are reusable, but they need call backs and update methods to function. It is possible o hardwire all this from the outside, but the number of links increases rapidly with the number of widgets on the screen.

## Centralized variables

By default, the variables represented eg by a slider are not linked to the other GUI elements. Below, we attempt an alternative approach where the widgets are linked by an object called a variable. The aim is to connect by default all elements called time for example. If a display needs two time variables, then these can be given different names.

In [ ]:
# Imports

# arrays
import xarray as xr
import numpy as np 
import zarr
# plotting
import matplotlib.pyplot as plt 
%matplotlib widget
plt.rcParams['figure.figsize'] = [10, 5]
# gui elements
import ipywidgets as widgets
from ipyleaflet import *

# 

In [ ]:
#define Variables


class Observer: #base class for observers
    def update(self,v):
        # you should override this function with something meaningful
        pass

class Variable:
    def __init__(self,name: str, label:str):
        self._name=name
        self._label=label
        self._observers=[]
        self._value=None
    def observe(self,observer:Observer): 
        # an observer is a function that is called when the variable is changed
        # the only argument is the variable
        self._observers.append(observer)
    @property
    def name(self): # to allow for temp=variable.name
        return self._name
    #make immutable
    #@name.setter 
    #def name(self,new_name: str): # allow for variale.name=new_name
    #    self._name=new_name
    @property
    def label(self):
        return self._label
    #@label.setter
    #def label(self,new_label: str):
    #    self._label=new_label

class IntVariable(Variable):
    def __init__(self,name: str,label: str, min_val:int, max_val:int, initial_val:int):
        super().__init__(name,label)
        self._min=min_val
        self._max=max_val
        self._value=initial_val
    @property
    def value(self):
        return self._value
    @value.setter
    def value(self,new_value: int):
        print(f"set value of {self.name} to {new_value}\n")
        #truncate to range
        v=max(new_value,self._min)
        v=min(v,self._max)
        self._value=v
        #inform observers
        for obs in self._observers:
            obs.update(self)
    @property
    def min(self):
        return self._min
    @property
    def max(self):
        return self._max

class ListVariable(Variable):
    def __init__(self,name: str,label: str, items=[]):
        super().__init__(name,label)
        self._items=items
        if len(items)<1:
            self._value=None
        else:
            self._value=0
    @property
    def value(self):
        return self._value
    @value.setter
    def value(self,new_value: int):
        print(f"set value of {self.name} to {new_value}\n")
        #truncate to range
        n=len(self._items)
        if n>0:
            value=max(new_value,0)
            value=min(value,n-1)
            self._value=value 
            #inform observers
            for obs in self._observers:
                obs.update(self)
    @property
    def value_str(self):
        if self._value==None:
            return ""
        else:
            res=self._items[self._value]
            return res
    @value_str.setter
    def value_str(self,new_value: str):
        print(f"set value of {self.name} to {new_value}\n")
        #truncate to range
        n=len(self._items)
        if n>0:
            if new_value in self._items:
                ivalue=self._items.index(new_value)
                self._value=ivalue 
                #inform observers
                for obs in self._observers:
                    obs.update(self)



In [ ]:
#demo of Variable class

# A variable with an integer in a range
v1=IntVariable("layer","Vertical layer number",1,10,1)
print(f'variable name={v1.name} label={v1.label}\n')
print(f'current {v1.name} has value {v1.value} of range {v1.min}:{v1.max}\n')

v1.value=3
print("change value\n")
print(f'current {v1.name} has value {v1.value} of range {v1.min}:{v1.max}\n')

print("---------------------------------\n")
# a variable with a selection from a list
v2=ListVariable("station","Station name",["Vlissingen","Hoek van Holland","Platform K13"])
print(f'variable name={v2.name} label={v2.label}\n')
print(f'current {v2.name} has value {v2.value} which denotes {v2.value_str} \n')

v2.value=1
print(f'current {v2.name} has value {v2.value} which denotes {v2.value_str} \n')

v2.value_str="Platform K13"
print(f'current {v2.name} has value {v2.value} which denotes {v2.value_str} \n')


In [ ]:
v2.value_str

In [ ]:
class WtIntSlider(widgets.IntSlider):
    def __init__(self,v: IntVariable, orientation='horizontal'):
        super().__init__(value=v.value, min=v.min, max=v.max, step=1,
        description=v.label, disabled=False, continuous_update=False,
        orientation=orientation) #, readout=True, readout_format='d' 
        self._variable=v
        self.observe(self.update_from_slider,names="value")
        v.observe(self)
    def update_from_slider(self,obj):
        print("signal from slider")
        self._variable.value=self.value
    def update(self,v: IntVariable):
        print(f"slider receives signal from variable {v.name}. Value is {v.value}\n")
        self.value=str(v.value)

class WtIntLabel(widgets.Label):
    def __init__(self,v: IntVariable, orientation='horizontal'):
        super().__init__(value=str(v.value), description=v.label, 
        disabled=False, orientation=orientation)
        self._variable=v
        v.observe(self)
    def update(self,v: IntVariable):
        print(f"signal from variable {v.name}. Value is {v.value}\n")
        self.value=str(v.value)

In [ ]:
# Demo for Int slider and label.
# The widgets are connected with the variable
v2=IntVariable("itime","Time-step",1,100,1)
sld2a=WtIntSlider(v2)
sld2b=WtIntSlider(v2)
lbl2=WtIntLabel(v2)
display(sld2a,sld2b,lbl2)

In [ ]:
dd1=widgets.Dropdown(
    options=["Cuxhaven","Brest"],
    value="Cuxhaven",
    description='Location:',
)
display(dd1)
lbl_dd1=widgets.Label(dd1.value)
display(lbl_dd1)
def update_label(dd):
    text=dd1.value
    lbl_dd1.value=text
dd1.observe(update_label,names="value")

In [ ]:
class WtListDropdown(widgets.Dropdown):
    def __init__(self,v: ListVariable, orientation='horizontal'):
        super().__init__(options=v._items, value=v.value_str,
        description=v.label, disabled=False, continuous_update=False,
        orientation=orientation)
        self._variable=v
        self.observe(self.update_from_dropdown,names="value")
        v.observe(self)
    def update_from_dropdown(self,obj):
        print("signal from dropdown")
        self._variable.value_str=self.value
    def update(self,v: ListVariable):
        print(f"dropdown receives signal from variable {v.name}. Value is {v.value_str}\n")
        if !(self.value==v.value_str):
            self.value=v.value_str

class WtListLabel(widgets.Label):
    def __init__(self,v: ListVariable, orientation='horizontal'):
        super().__init__(value=v.value_str, description=v.label, 
        disabled=False, orientation=orientation)
        self._variable=v
        v.observe(self)
    def update(self,v: ListVariable):
        print(f"signal from variable {v.name}. Value is {v.value}\n")
        self.value=v.value_str

In [ ]:
v3=ListVariable("station","Station name",["Vlissingen","Hoek van Holland","Platform K13"])
sld3a=WtListDropdown(v3)
sld3b=WtListDropdown(v3)
lbl3=WtListLabel(v3)
display(sld3a,sld3b,lbl3)

In [ ]:
v3.value=2